In [47]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [77]:
house_train = pd.read_csv('./MLproject/train.csv')
house_test = pd.read_csv('./MLProject/test.csv')
house_train.shape

(1460, 81)

In [49]:
house_test.shape

(1459, 80)

In [50]:
house_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1455 non-null object
LotFrontage      1232 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            107 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1457 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1458 non-

In [78]:
f = open('./MLProject/data_description.txt', 'r')
rawlines = f.readlines()
f.close()
len(rawlines)

523

In [79]:
lines = list(map(str.strip, rawlines))
lines = list(filter(None, lines))
print(len(lines))
print(lines[:5])
             

399
['MSSubClass: Identifies the type of dwelling involved in the sale.', '20\t1-STORY 1946 & NEWER ALL STYLES', '30\t1-STORY 1945 & OLDER', '40\t1-STORY W/FINISHED ATTIC ALL AGES', '45\t1-1/2 STORY - UNFINISHED ALL AGES']


In [80]:
import re
rep_string = re.compile(':.*')
house_categories = {}
for line in lines:
    if '\t' not in line and ': ' in line:
        key = rep_string.sub('', line)
    else:
        value = line.split('\t')[0]
        if key not in house_categories:  
            house_categories[key] = [value]
        else:
            house_categories[key].append(value)     


In [54]:
len(house_categories.keys())

46

In [81]:
for varname in house_categories.keys():
    diff = []
    diff = set(house_train[varname].unique().astype(str)).difference(set(house_categories[varname]))
    if len(diff) and diff != {'nan'}:
        print(varname)
        print('-'*10)
        print(diff)
        print('\n')

MSZoning
----------
{'C (all)'}


Neighborhood
----------
{'NAmes'}


BldgType
----------
{'Twnhs', '2fmCon', 'Duplex'}


Exterior2nd
----------
{'CmentBd', 'Wd Shng', 'Brk Cmn'}


SaleType
----------
{'WD'}




In [82]:
house_categories['MSZoning'][1] = 'C (all)'
house_categories['MSZoning']

['A', 'C (all)', 'FV', 'I', 'RH', 'RL', 'RP', 'RM']

In [83]:
list(enumerate(house_categories['Neighborhood']))
house_categories['Neighborhood'][12] = 'NAmes'

In [84]:
house_categories['BldgType']

['1Fam', '2FmCon', 'Duplx', 'TwnhsE', 'TwnhsI']

In [85]:
house_train['BldgType'].unique()

array(['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'], dtype=object)

In [86]:
house_categories['BldgType'][1] = '2fmCon'
house_categories['BldgType'][2] = 'Duplex'
house_categories['BldgType'][4] = 'Twnhs'
house_categories['BldgType']

['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs']

In [87]:
list(enumerate(house_categories['Exterior2nd']))

[(0, 'AsbShng'),
 (1, 'AsphShn'),
 (2, 'BrkComm'),
 (3, 'BrkFace'),
 (4, 'CBlock'),
 (5, 'CemntBd'),
 (6, 'HdBoard'),
 (7, 'ImStucc'),
 (8, 'MetalSd'),
 (9, 'Other'),
 (10, 'Plywood'),
 (11, 'PreCast'),
 (12, 'Stone'),
 (13, 'Stucco'),
 (14, 'VinylSd'),
 (15, 'Wd Sdng'),
 (16, 'WdShing')]

In [88]:
house_categories['Exterior2nd'][2] = 'Brk Cmn'
house_categories['Exterior2nd'][5] = 'CmentBd'
house_categories['Exterior2nd'][16] = 'Wd Shng'
house_categories['Exterior2nd']

['AsbShng',
 'AsphShn',
 'Brk Cmn',
 'BrkFace',
 'CBlock',
 'CmentBd',
 'HdBoard',
 'ImStucc',
 'MetalSd',
 'Other',
 'Plywood',
 'PreCast',
 'Stone',
 'Stucco',
 'VinylSd',
 'Wd Sdng',
 'Wd Shng']

In [89]:
house_categories['SaleType'][0]='WD'
house_categories['SaleType']

['WD', 'CWD', 'VWD', 'New', 'COD', 'Con', 'ConLw', 'ConLI', 'ConLD', 'Oth']

In [90]:
for varname in house_categories.keys():
    diff = []
    diff = set(house_train[varname].unique().astype(str)).difference(set(house_categories[varname]))
    if len(diff) and diff != {'nan'}:
        print(varname)
        print('-'*10)
        print(diff)
        print('\n')

In [91]:
train_id = house_train['Id']
test_id = house_test['Id']

house_train.drop("Id", axis = 1, inplace = True)
house_test.drop("Id", axis = 1, inplace = True)

In [92]:
y_train = house_train['SalePrice']
X_train = house_train.drop('SalePrice', axis=1)
X_test = house_test.copy()

In [93]:
del house_train, house_test

In [101]:
all_data = pd.concat([X_train, X_test], ignore_index=True)
all_data.shape

(2919, 79)

In [102]:
all_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [103]:
all_data.YrSold.dtype

dtype('int64')

In [104]:
from MLProject.preprocess import impute

In [105]:
all_data = impute(all_data)

In [106]:
print(all_data.YrSold.dtype)
all_data.head()

int64


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalSF
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,2566.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,2524.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,2706.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,2473.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,3343.0


In [107]:
from MLProject.featureEng import LabelCountEncoder

In [108]:
ord_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposureb' 'HeatingQC', 'KitchenQual', 
           'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
ord_dic = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa':2, 'Po':1, 'Av': 3, 'Mn': 2, 'Po': 1, 'None': 0}
ord2_cols = ['BsmtFinType1', 'BsmtFinType2', 'Fence']
ord2_dic = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'None': 0,
            'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw':1}

In [109]:
train_index = len(X_train)
for col in all_data.columns:
    if col in ord_cols:
        all_data[col] = all_data[col].map(lambda x: ord_dic.get(x, 0))
    elif col in ord2_cols:
        all_data[col] = all_data[col].map(lambda x: ord2_dic.get(x, 0))
    elif all_data[col].dtype == "object":
        lce = LabelCountEncoder()
        all_data.loc[:train_index, col] = lce.fit_transform(all_data.loc[:train_index, col])
        all_data.loc[train_index:, col] = lce.transform(all_data.loc[train_index:, col])

In [110]:
pd.set_option('display.max_columns', 100)
all_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalSF
0,14,5,65.0,8450,2,3,4,4,5,3,24,9,8,5,7,7,9,2003,2003,6,8,15,16,3,196.0,4,3,6,4,3,5,6,706.0,1,0.0,150.0,856.0,6,5,2,5,856,854,0,1710,1.0,0.0,2,1,3,1,4,8,7,0,0,7,2003.0,3,2.0,548.0,3,3,3,0,61,0,0,0,0,0,0,5,0,2,2008,9,6,2566.0
1,15,5,80.0,9600,2,3,4,4,2,3,3,8,8,5,8,6,6,1976,1976,6,8,13,15,4,0.0,3,3,5,4,3,3,5,978.0,1,0.0,284.0,1262.0,6,5,2,5,1262,0,0,1262,0.0,1.0,2,0,3,1,3,6,7,1,3,7,1976.0,3,2.0,460.0,3,3,3,298,0,0,0,0,0,0,0,5,0,5,2007,9,6,2524.0
2,14,5,68.0,11250,2,3,3,4,5,3,24,9,8,5,7,7,9,2001,2002,6,8,15,16,3,162.0,4,3,6,4,3,2,6,486.0,1,0.0,434.0,920.0,6,5,2,5,920,866,0,1786,1.0,0.0,2,1,3,1,4,6,7,1,3,7,2001.0,3,2.0,608.0,3,3,3,0,42,0,0,0,0,0,0,5,0,9,2008,9,6,2706.0
3,9,5,60.0,9550,2,3,3,4,4,3,14,9,8,5,7,7,9,1915,1970,6,8,12,10,4,0.0,3,3,4,3,4,5,5,216.0,1,0.0,540.0,756.0,6,3,2,5,961,756,0,1717,1.0,0.0,1,0,3,1,4,7,7,1,4,6,1998.0,4,3.0,642.0,3,3,3,0,35,272,0,0,0,0,0,5,0,2,2006,9,4,2473.0
4,14,5,84.0,14260,2,3,3,4,2,3,12,9,8,5,7,8,9,2000,2000,6,8,15,16,3,350.0,4,3,6,4,3,4,6,655.0,1,0.0,490.0,1145.0,6,5,2,5,1145,1053,0,2198,1.0,0.0,2,1,4,1,4,9,7,1,3,7,2000.0,3,3.0,836.0,3,3,3,192,84,0,0,0,0,0,0,5,0,12,2008,9,6,3343.0


Adding interaction features

In [111]:
all_data['Nbrhd_QC'] = all_data.Neighborhood + all_data.OverallQual + all_data.OverallCond
all_data['Nbrhd_Area'] = all_data.Neighborhood + all_data.LotArea
all_data['Nbrhd_SF'] = all_data.Neighborhood + all_data.TotalSF
all_data['Nbrhd_SFQC'] = all_data.Neighborhood + all_data.TotalSF + all_data.OverallQual + all_data.OverallCond
all_data['Yr_Sales'] = all_data.YrSold + all_data.SaleType + all_data.SaleCondition

In [112]:
X_train = all_data.iloc[:train_index, :]
X_test = all_data.iloc[train_index:, :]

In [113]:
X_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalSF,Nbrhd_QC,Nbrhd_Area,Nbrhd_SF,Nbrhd_SFQC,Yr_Sales
1460,0,0,80.0,11622,0,0,0,0,0,0,0,0,0,0,0,5,0,1961,1961,0,0,0,0,0,0.0,3,3,0,3,3,0,3,468.0,2,144.0,270.0,882.0,0,0,0,0,896,0,0,896,0.0,0.0,1,0,2,1,3,5,0,0,0,0,1961.0,0,1.0,730.0,3,3,0,140,0,0,0,120,0,0,3,0,0,6,2010,0,0,1778.0,5,11622,1778.0,1783.0,2010
1461,15,5,81.0,14267,2,3,3,4,4,3,25,9,8,5,8,6,8,1958,1958,5,8,12,13,3,108.0,3,3,5,3,3,5,5,923.0,1,0.0,406.0,1329.0,6,4,2,5,1329,0,0,1329,0.0,0.0,1,1,3,1,4,6,7,0,0,7,1958.0,4,1.0,312.0,3,3,3,393,36,0,0,0,0,0,0,3,12500,6,2010,9,6,2658.0,39,14292,2683.0,2697.0,2025
1462,14,5,74.0,13830,2,3,3,4,5,3,20,9,8,5,7,5,9,1997,1998,6,8,15,16,4,0.0,3,3,6,4,3,5,6,791.0,1,0.0,137.0,928.0,6,3,2,5,928,701,0,1629,0.0,0.0,2,1,3,1,3,6,7,1,3,7,1997.0,2,2.0,482.0,3,3,3,212,34,0,0,0,0,0,3,5,0,3,2010,9,6,2557.0,34,13850,2577.0,2591.0,2025
1463,14,5,78.0,9978,2,3,3,4,5,3,20,9,8,5,7,6,8,1998,1998,6,8,15,16,3,20.0,3,3,6,3,3,5,6,602.0,1,0.0,324.0,926.0,6,5,2,5,926,678,0,1604,0.0,0.0,2,1,3,1,4,7,7,1,4,7,1998.0,2,2.0,470.0,3,3,3,360,36,0,0,0,0,0,0,5,0,6,2010,9,6,2530.0,34,9998,2550.0,2564.0,2025
1464,12,5,43.0,5005,2,3,3,2,5,3,8,9,8,4,8,8,9,1992,1992,6,8,14,14,4,0.0,4,3,6,4,3,5,5,263.0,1,0.0,1017.0,1280.0,6,5,2,5,1280,0,0,1280,0.0,0.0,2,0,2,1,4,5,7,0,0,7,1992.0,3,2.0,506.0,3,3,3,0,82,0,0,144,0,0,0,5,0,1,2010,9,6,2560.0,25,5013,2568.0,2585.0,2025


In [158]:
X_test.shape

(1459, 84)

In [159]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 1460 to 2918
Data columns (total 84 columns):
MSSubClass       1459 non-null int64
MSZoning         1459 non-null int64
LotFrontage      1459 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null int64
Alley            1459 non-null int64
LotShape         1459 non-null int64
LandContour      1459 non-null int64
LotConfig        1459 non-null int64
LandSlope        1459 non-null int64
Neighborhood     1459 non-null int64
Condition1       1459 non-null int64
Condition2       1459 non-null int64
BldgType         1459 non-null int64
HouseStyle       1459 non-null int64
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null int64
RoofMatl         1459 non-null int64
Exterior1st      1459 non-null int64
Exterior2nd      1459 non-null int64
MasVnrType       1459 non-null int64


### Statsmodel OLS

In [114]:
import statsmodels.api as sm

C:\Users\Gregory\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [140]:
model = sm.OLS(y_train.transform(np.log1p), X_train.transform(np.log1p)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.417e+05
Date:                Mon, 05 Mar 2018   Prob (F-statistic):               0.00
Time:                        00:24:03   Log-Likelihood:                 914.48
No. Observations:                1460   AIC:                            -1661.
Df Residuals:                    1376   BIC:                            -1217.
Df Model:                          84                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
MSSubClass       -0.0057      0.020     -0.291      0.771      -0.044       0.033
MSZoning          0.1144      0.030      3.862      0.000       0.056       0.173
LotFrontage       0.0174      0.018      0.992      0.322      -0.017       0.052
LotArea           3.4033      6.826      0.499      0.618      -9.986      16.793
Street            0.2730      0.149      1.827      0.068      -0.020       0.566
Alley            -0.0627      0.034     -1.870      0.062      -0.128       0.003
LotShape          0.0281      0.029      0.975      0.330      -0.028       0.085
LandContour      -0.0074      0.028     -0.264      0.792      -0.062       0.048
LotConfig        -0.0085      0.022     -0.381      0.704      -0.052       0.035
LandSlope        -0.1103      0.056     -1.978      0.048      -0.220      -0.001
Neighborhood      0.0693      0.053      1.296      0.195      -0.036       0.174
Condition1        0.0835      0.024      3.438      0.001       0.036       0.131
Condition2        0.1090      0.062      1.752      0.080      -0.013       0.231
BldgType          0.0079      0.025      0.322      0.748      -0.040       0.056
HouseStyle       -0.0296      0.028     -1.039      0.299      -0.085       0.026
OverallQual       0.5446      0.055      9.843      0.000       0.436       0.653
OverallCond       0.1296      0.052      2.510      0.012       0.028       0.231
YearBuilt         0.7659      0.576      1.331      0.183      -0.363       1.895
YearRemodAdd      3.0911      0.544      5.680      0.000       2.024       4.159
RoofStyle        -0.0603      0.036     -1.664      0.096      -0.131       0.011
RoofMatl          0.3058      0.064      4.811      0.000       0.181       0.431
Exterior1st      -0.0180      0.029     -0.610      0.542      -0.076       0.040
Exterior2nd       0.0182      0.026      0.701      0.483      -0.033       0.069
MasVnrType       -0.0279      0.036     -0.776      0.438      -0.098       0.043
MasVnrArea       -0.0025      0.003     -1.001      0.317      -0.007       0.002
ExterQual         0.1938      0.050      3.869      0.000       0.096       0.292
ExterCond         0.0512      0.047      1.085      0.278      -0.041       0.144
Foundation        0.0509      0.036      1.414      0.158      -0.020       0.121
BsmtQual          0.1044      0.042      2.500      0.013       0.022       0.186
BsmtCond          0.0922      0.053      1.751      0.080      -0.011       0.196
BsmtExposure     -0.0440      0.019     -2.377      0.018      -0.080      -0.008
BsmtFinType1      0.0045      0.019      0.240      0.810      -0.032       0.041
BsmtFinSF1        0.0073      0.004      2.059      0.040       0.000       0.014
BsmtFinType2      0.0302      0.038      0.801      0.423      -0.044       0.104
BsmtFinSF2       -0.0073      0.005     -1.340   

In [304]:
coef_ = pd.DataFrame(model.params,columns=['coef'])


### OLS with significant features

In [292]:
sigFeature = [('MSZoning', .1144),('LandSlope', -.1103), ('Condition1', .0835),
				 ('OverallQual', .5446), ('OverallCond', .1296), ('YearRemodAdd', 3.0911),
				 ('RoofMatl', .3058), ('ExterQual', .1938), ('BsmtQual', .1044), ('BsmtExposure', -.0440),
				 ('BsmtFinSF1', .0073), ('BsmtUnfSF', -.0065), ('TotalBsmtSF', -.0497), ('CentralAir', .01537),
				 ('1stFlrSF', -.0982), ('2ndFlrSF', -.0135), ('GrLivArea', .2235), ('BsmtFullBath', .0299),
				 ('FullBath', .0604), ('HalfBath', .0443), ('BedroomAbvGr', -.0694), ('KitchenAbvGr', -.1927),
				 ('KitchenQual', .1282), ('TotRmsAbvGrd', .0811), ('Functional', .2338), ('GarageYrBlt', -.0472),
				 ('GarageCars', .0980), ('ScreenPorch', .0077), ('TotalSF', -22.0876), ('Nbrhd_QC', -.4555),
				 ('Nbrhd_SF', 34.3477)]

In [293]:
ord_sigFeat = [(x,abs(y))for (x,y) in sigFeature]

In [294]:
ord_sigFeat = sorted(ord_sigFeat,key=lambda x: x[1], reverse=True)

In [295]:
len(ord_sigFeat)

31

In [296]:
sigFeat31 = [x for (x,y) in ord_sigFeat]
sigFeat12 = [x for (x,y) in ord_sigFeat[:12]]

In [377]:
X2 = X_train.loc[:,sigFeat12] 

In [378]:
print(y_train.shape)
print(X2.shape)

(1460,)
(1460, 12)


In [342]:
X2.shape[0]

1460

In [379]:
X2cons = X2.copy()
X2cons['cons'] = np.ones(1460)

In [380]:
X2cons.drop('RoofMatl', axis=1, inplace=True)

In [383]:
X2cons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 12 columns):
Nbrhd_SF        1460 non-null float64
TotalSF         1460 non-null float64
YearRemodAdd    1460 non-null int64
OverallQual     1460 non-null int64
Nbrhd_QC        1460 non-null int64
Functional      1460 non-null int64
GrLivArea       1460 non-null int64
ExterQual       1460 non-null int64
KitchenAbvGr    1460 non-null int64
OverallCond     1460 non-null int64
KitchenQual     1460 non-null int64
cons            1460 non-null float64
dtypes: float64(3), int64(9)
memory usage: 137.0 KB


In [384]:
model_ = sm.OLS(y_train.transform(np.log1p), X2cons.transform(np.log1p)).fit()
model_.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.836
Model:                            OLS   Adj. R-squared:                  0.835
Method:                 Least Squares   F-statistic:                     671.6
Date:                Tue, 06 Mar 2018   Prob (F-statistic):               0.00
Time:                        13:32:52   Log-Likelihood:                 588.90
No. Observations:                1460   AIC:                            -1154.
Df Residuals:                    1448   BIC:                            -1090.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Nbrhd_SF        36.8637      3.665     10.059      0.000      29.675      44.052
TotalSF        -36.0303      3.629     -9.930      0.000     -43.148     -28.912
YearRemodAdd     4.9219      0.552      8.919      0.000       3.839       6.004
OverallQual      0.5827      0.040     14.715      0.000       0.505       0.660
Nbrhd_QC        -0.4775      0.052     -9.246      0.000      -0.579      -0.376
Functional       0.2758      0.039      7.143      0.000       0.200       0.352
GrLivArea        0.1433      0.027      5.326      0.000       0.091       0.196
ExterQual        0.2247      0.055      4.107      0.000       0.117       0.332
KitchenAbvGr    -0.3350      0.051     -6.559      0.000      -0.435      -0.235
OverallCond      0.1940      0.027      7.253      0.000       0.142       0.247
KitchenQual      0.2240      0.046      4.865      0.000       0.134       0.314
cons           -49.1518      6.013     -8.175      0.000     -60.946     -37.357
==============================================================================
Omnibus:                      678.317   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11049.962
Skew:                          -1.743   Prob(JB):                         0.00
Kurtosis:                      16.019   Cond. No.                     2.32e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [401]:
model_.wald_test(model_.params)

array([[ 150.51402531]])

In [404]:
X3 = X2cons.drop(['Nbrhd_SF', 'Nbrhd_QC'], axis=1)

In [405]:
X3['Neighborhood'] = X_train.loc[:,'Neighborhood']

In [406]:
model3 = sm.OLS(y_train.transform(np.log1p), X3.transform(np.log1p)).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.825
Model:                            OLS   Adj. R-squared:                  0.823
Method:                 Least Squares   F-statistic:                     681.5
Date:                Tue, 06 Mar 2018   Prob (F-statistic):               0.00
Time:                        14:06:33   Log-Likelihood:                 539.58
No. Observations:                1460   AIC:                            -1057.
Df Residuals:                    1449   BIC:                            -999.0
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
TotalSF          0.4697      0.030     15.526      0.000       0.410       0.529
YearRemodAdd     4.6567      0.570      8.169      0.000       3.538       5.775
OverallQual      0.4523      0.039     11.686      0.000       0.376       0.528
Functional       0.2731      0.040      6.841      0.000       0.195       0.351
GrLivArea        0.1971      0.027      7.233      0.000       0.144       0.251
ExterQual        0.2855      0.056      5.079      0.000       0.175       0.396
KitchenAbvGr    -0.3300      0.053     -6.255      0.000      -0.434      -0.227
OverallCond      0.0723      0.024      3.007      0.003       0.025       0.119
KitchenQual      0.2578      0.047      5.429      0.000       0.165       0.351
cons           -44.1475      6.197     -7.124      0.000     -56.304     -31.991
Neighborhood    -0.0011      0.011     -0.104      0.917      -0.022       0.020
==============================================================================
Omnibus:                      480.068   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4574.193
Skew:                          -1.250   Prob(JB):                         0.00
Kurtosis:                      11.303   Cond. No.                     1.99e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.99e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Corrected X2 model
#### Attempt to correct collinearity in features

In [441]:
X4 = X2.drop(['TotalSF', 'Nbrhd_QC', 'RoofMatl'], axis = 1)
X4['cons']=np.ones(X4.shape[0])

In [442]:
model4 = sm.OLS(y_train.transform(np.log1p), X4.transform(np.log1p)).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.825
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     759.1
Date:                Tue, 06 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:08:15   Log-Likelihood:                 540.71
No. Observations:                1460   AIC:                            -1061.
Df Residuals:                    1450   BIC:                            -1009.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Nbrhd_SF         0.4760      0.030     15.625      0.000       0.416       0.536
YearRemodAdd     4.6567      0.569      8.181      0.000       3.540       5.773
OverallQual      0.4530      0.039     11.743      0.000       0.377       0.529
Functional       0.2732      0.040      6.852      0.000       0.195       0.351
GrLivArea        0.1957      0.027      7.203      0.000       0.142       0.249
ExterQual        0.2840      0.056      5.058      0.000       0.174       0.394
KitchenAbvGr    -0.3314      0.053     -6.302      0.000      -0.435      -0.228
OverallCond      0.0721      0.024      3.002      0.003       0.025       0.119
KitchenQual      0.2565      0.047      5.411      0.000       0.164       0.350
cons           -44.2076      6.189     -7.143      0.000     -56.348     -32.067
==============================================================================
Omnibus:                      483.554   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4662.005
Skew:                          -1.257   Prob(JB):                         0.00
Kurtosis:                      11.385   Cond. No.                     1.95e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.95e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Sparse OLS model

In [409]:
Xlst = X_train.loc[:,['Neighborhood', 'TotalSF', 'OverallQual', 'OverallCond']]
Xlst['cons'] = np.ones(1460)
model = sm.OLS(y_train.transform(np.log1p), Xlst.transform(np.log1p)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     1254.
Date:                Tue, 06 Mar 2018   Prob (F-statistic):               0.00
Time:                        14:25:23   Log-Likelihood:                 357.87
No. Observations:                1460   AIC:                            -705.7
Df Residuals:                    1455   BIC:                            -679.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Neighborhood     0.0041      0.012      0.344      0.731      -0.019       0.027
TotalSF          0.6258      0.021     29.152      0.000       0.584       0.668
OverallQual      0.9066      0.035     26.088      0.000       0.838       0.975
OverallCond      0.0794      0.027      2.974      0.003       0.027       0.132
cons             7.4975      0.209     35.794      0.000       7.087       7.908
==============================================================================
Omnibus:                      324.527   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1844.110
Skew:                          -0.911   Prob(JB):                         0.00
Kurtosis:                       8.196   Cond. No.                         376.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### K-fold validated OLS

In [369]:
model2 = lr(normalize=True)
k5 = ms.KFold(n_splits=5)
train_error = []
test_error = []
for train_index, test_index in k5.split(y_train):
    x1_train = X2.iloc[train_index,:]
    y1_train = y_train[train_index]
    x1_test = X2.iloc[test_index,:]
    y1_test = y_train[test_index]
    ## Fit a model with the training dataset
    print(x1_train.shape, y1_train.shape)
    model2.fit(x1_train.transform(np.log1p), y1_train.transform(np.log1p))
    train_error.append(model2.score(x1_train.transform(np.log1p), y1_train.transform(np.log1p)))
    test_error.append(model2.score(x1_test.transform(np.log1p), y1_test.transform(np.log1p)))
    
## Find the training error
print('Train error: {}'.format(1 - np.array(train_error)))

## Find the test error 
print('Test error: {}'.format(1 - np.array(test_error)))

(1168, 12) (1168,)
(1168, 12) (1168,)
(1168, 12) (1168,)
(1168, 12) (1168,)
(1168, 12) (1168,)
Train error: [ 0.16024162  0.16245894  0.17303213  0.16486124  0.14822315]
Test error: [ 0.22007057  0.1684149   0.12979405  0.16021264  0.25040794]


### X2 w/ constant Kfold CV

In [388]:
model2cons = lr(normalize=True)
k5c = ms.KFold(n_splits=5)
ctrain_error = []
ctest_error = []
for train_index, test_index in k5c.split(y_train):
    x1c_train = X2cons.iloc[train_index,:]
    y1c_train = y_train[train_index]
    x1c_test = X2cons.iloc[test_index,:]
    y1c_test = y_train[test_index]
    ## Fit a model with the training dataset
    model2cons.fit(x1c_train.transform(np.log1p), y1c_train.transform(np.log1p))
    ctrain_error.append(model2cons.score(x1c_train.transform(np.log1p), y1c_train.transform(np.log1p)))
    ctest_error.append(model2cons.score(x1c_test.transform(np.log1p), y1c_test.transform(np.log1p)))
    
## Find the training error
print('Train error: {}'.format(1 - np.array(ctrain_error)))
print(ctrain_error)
## Find the test error 
print('')
print('Test error: {}'.format(1 - np.array(ctest_error)))
print(ctest_error)

Train error: [ 0.16646853  0.16316564  0.17352196  0.16530255  0.14949126]
[0.83353146978579884, 0.8368343625112904, 0.82647804112203738, 0.83469745017593033, 0.85050873834829388]

Test error: [ 0.1533983   0.16774655  0.12995417  0.160461    0.228726  ]
[0.84660169779499472, 0.83225344821132796, 0.87004582891421745, 0.83953900025727513, 0.7712739998916005]


In [390]:
X2cons_test = X_test.loc[:,sigFeat12]
X2cons_test['cons'] = np.ones(X2cons_test.shape[0])
X2cons_test.drop('RoofMatl', axis=1, inplace=True)

In [431]:
y_pred_OLS1 = pd.DataFrame(model2cons.predict(X2cons_test.transform(np.log1p)), index = X_test.index, columns = ['SalePrice']).transform(np.expm1)

In [432]:
y_pred_OLS1.set_index(y_pred_OLS1.index + 1, inplace=True)
y_pred_OLS1 = y_pred_OLS1.rename_axis('Id')
y_pred_OLS1.head()

,SalePrice
Id,
1461,61947.239330
1462,161515.989242
1463,162517.795391
1464,179696.178821
1465,204640.138337


In [433]:
y_pred_OLS1.to_csv('y_pred_ols1.csv')

### X3 model KFold CV (modified X2cons)

In [410]:
model3 = lr(normalize=True)
k53 = ms.KFold(n_splits=5)
train3_error = []
test3_error = []
for train_index, test_index in k5c.split(y_train):
    x3_train = X3.iloc[train_index,:]
    y3_train = y_train[train_index]
    x3_test = X3.iloc[test_index,:]
    y3_test = y_train[test_index]
    ## Fit a model with the training dataset
    model3.fit(x3_train.transform(np.log1p), y3_train.transform(np.log1p))
    train3_error.append(model3.score(x3_train.transform(np.log1p), y3_train.transform(np.log1p)))
    test3_error.append(model3.score(x3_test.transform(np.log1p), y3_test.transform(np.log1p)))
    
## Find the training error
print('Train error: {}'.format(1 - np.array(train3_error)))
print(train3_error)
## Find the test error 
print('')
print('Test error: {}'.format(1 - np.array(test3_error)))
print(test3_error)

Train error: [ 0.17848711  0.171527    0.18409753  0.17735445  0.16380207]
[0.82151289455482035, 0.8284729970515482, 0.81590246992975179, 0.82264554508895238, 0.83619792873806875]

Test error: [ 0.16247412  0.19005265  0.14480289  0.16931553  0.2265858 ]
[0.83752588106086756, 0.80994735290114661, 0.85519711293965384, 0.8306844674216437, 0.77341420450835507]


### X4 OLS 
Removed collinear features from X2

In [443]:
model4 = lr(normalize=True)
k5_4 = ms.KFold(n_splits=5)
train4_error = []
test4_error = []
for train_index, test_index in k5_4.split(y_train):
    x4_train = X4.iloc[train_index,:]
    y4_train = y_train[train_index]
    x4_test = X4.iloc[test_index,:]
    y4_test = y_train[test_index]
    ## Fit a model with the training dataset
    model4.fit(x4_train.transform(np.log1p), y4_train.transform(np.log1p))
    train4_error.append(model4.score(x4_train.transform(np.log1p), y4_train.transform(np.log1p)))
    test4_error.append(model4.score(x4_test.transform(np.log1p), y4_test.transform(np.log1p)))
    
## Find the training error
print('Train error: {}'.format(1 - np.array(train4_error)))
print(train4_error)
## Find the test error 
print('')
print('Test error: {}'.format(1 - np.array(test4_error)))
print(test4_error)

Train error: [ 0.17819957  0.17134907  0.18383151  0.17727346  0.16345887]
[0.82180042605191139, 0.82865093280177227, 0.81616848911353046, 0.82272653796077555, 0.83654112988941542]

Test error: [ 0.16225337  0.18941846  0.14416331  0.16752642  0.22662927]
[0.83774662748325135, 0.81058153624765095, 0.85583669112182359, 0.83247358128202464, 0.77337073114008825]


In [450]:
X4_test = X_test.loc[:,X4.columns[:-1]]
X4_test['cons'] = np.ones(X4_test.shape[0])

(1459, 10)

In [452]:
y_pred_OLS4 = pd.DataFrame(model4.predict(X4_test.transform(np.log1p)), index = X_test.index, columns = ['SalePrice']).transform(np.expm1)

In [453]:
y_pred_OLS4.set_index(y_pred_OLS4.index + 1, inplace=True)
y_pred_OLS4 = y_pred_OLS4.rename_axis('Id')
y_pred_OLS4.head()

,SalePrice
Id,
1461,53351.941239
1462,162678.910461
1463,165855.609104
1464,183737.633397
1465,205879.077439


In [454]:
y_pred_OLS4.to_csv('y_pred_ols4.csv')

### Sparse model Kfold CV

In [411]:
model2l = lr(normalize=True)
k5l = ms.KFold(n_splits=5)
ltrain_error = []
ltest_error = []
for train_index, test_index in k5l.split(y_train):
    x1l_train = Xlst.iloc[train_index,:]
    y1l_train = y_train[train_index]
    x1l_test = Xlst.iloc[test_index,:]
    y1l_test = y_train[test_index]
    ## Fit a model with the training dataset
    model2l.fit(x1l_train.transform(np.log1p), y1l_train.transform(np.log1p))
    ltrain_error.append(model2l.score(x1l_train.transform(np.log1p), y1l_train.transform(np.log1p)))
    ltest_error.append(model2l.score(x1l_test.transform(np.log1p), y1l_test.transform(np.log1p)))
    
## Find the training error
print('Train error: {}'.format(1 - np.array(ltrain_error)))

## Find the test error 
print('Test error: {}'.format(1 - np.array(ltest_error)))

Train error: [ 0.22868818  0.21960052  0.22963047  0.22790871  0.21754028]
Test error: [ 0.20901006  0.24389718  0.2098635   0.2143546   0.25752779]


## Elasticnet
wherein I attempt an Elasticnet (Ridge+Lasso) model

In [413]:
en = ElasticNet()

In [422]:
en.set_params(alpha=3, fit_intercept=False, normalize=True)
en.fit(X2, y_train)

C:\Users\Gregory\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


ElasticNet(alpha=3, copy_X=True, fit_intercept=False, l1_ratio=0.5,
      max_iter=1000, normalize=True, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [424]:
las1 = lacv(fit_intercept=False, cv=5, normalize=True)
rid1 = rcv(fit_intercept=False, cv=5, normalize=True, )

In [427]:
las1.fit(X3, y_train)

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=False,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=True, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [428]:
las1.score(X3,y_train)

0.60919178611695968

In [429]:
rid1.fit(X3,y_train)

RidgeCV(alphas=(0.1, 1.0, 10.0), cv=5, fit_intercept=False, gcv_mode=None,
    normalize=True, scoring=None, store_cv_values=False)

In [425]:
ek5 = ms.KFold(n_splits=5)
etrain_error = []
etest_error = []
for train_index, test_index in ek5.split(y_train):
    xe_train = X3.iloc[train_index,:]
    ye_train = y_train[train_index]
    xe_test = X3.iloc[test_index,:]
    ye_test = y_train[test_index]
    ## Fit a model with the training dataset
    en.fit(xe_train.transform(np.log1p), ye_train.transform(np.log1p))
    etrain_error.append(en.score(xe_train.transform(np.log1p), ye_train.transform(np.log1p)))
    etest_error.append(en.score(xe_test.transform(np.log1p), ye_test.transform(np.log1p)))
    
## Find the training error
print('Train error: {}'.format(1 - np.array(etrain_error)))

## Find the test error 
print('Test error: {}'.format(1 - np.array(etest_error)))

TypeError: fit() missing 1 required positional argument: 'y'

## SK Regressions

In [423]:
from Kaggle_Workflow.stacking import stacking_regression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet, LinearRegression as lr, LassoCV as lacv, RidgeCV as rcv
from sklearn.ensemble import GradientBoostingRegressor as gbr, RandomForestRegressor as rfr

In [149]:
from sklearn import model_selection as ms
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
randomForest = ensemble.RandomForestClassifier()

### Random Forest

In [160]:
randomForest.set_params(n_estimators=100, random_state=42)
randomForest.fit(X_train, y_train)
print("The training error of random forest is: %.5f" %(1 - randomForest.score(X_train, y_train)))

The training error of random forest is: 0.00000


In [162]:
feature_importance = list(zip(X_train.columns, randomForest.feature_importances_))
dtype = [('feature', 'S10'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
feature_sort

array([(b'BsmtUnfSF',  0.03042456), (b'LotArea',  0.03003835),
       (b'Nbrhd_Area',  0.02996498), (b'GarageArea',  0.02978021),
       (b'GrLivArea',  0.02960723), (b'1stFlrSF',  0.0294496 ),
       (b'TotalSF',  0.02884797), (b'Nbrhd_SF',  0.02856624),
       (b'Nbrhd_SFQC',  0.0282533 ), (b'LotFrontag',  0.02825167),
       (b'TotalBsmtS',  0.02660247), (b'YearBuilt',  0.02616885),
       (b'BsmtFinSF1',  0.02592672), (b'YearRemodA',  0.02543996),
       (b'MoSold',  0.02530339), (b'GarageYrBl',  0.02372529),
       (b'Nbrhd_QC',  0.02252942), (b'OpenPorchS',  0.02153908),
       (b'WoodDeckSF',  0.02015964), (b'Yr_Sales',  0.0197629 ),
       (b'MasVnrArea',  0.01826524), (b'Neighborho',  0.01811354),
       (b'2ndFlrSF',  0.01643862), (b'TotRmsAbvG',  0.01623604),
       (b'Exterior2n',  0.01622797), (b'YrSold',  0.01574657),
       (b'Exterior1s',  0.01472464), (b'OverallQua',  0.01420194),
       (b'BsmtFinTyp',  0.01310565), (b'OverallCon',  0.0121498 ),
       (b'MSSubClass',

In [212]:
Imp_feat = [i.decode('utf-8') for (i, j) in feature_sort[0:20]]
numImp = list(enumerate(Imp_feat))

In [196]:
X_train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'O

In [201]:
missImp = [col for col in Imp_feat if col not in list(X_train.columns)]

In [203]:
ImpReplace = [i for (i,j) in numImp if j in missImp]

In [217]:
RealNames = ['LotFrontage', 'TotalBsmtSF', 'YearRemodAdd', 'GarageYrBlt', 'OpenPorchSF']

In [218]:
for i in range(len(ImpReplace)):
    Imp_feat[ImpReplace[i]] = RealNames[i]
print(Imp_feat)
[col for col in Imp_feat if col not in list(X_train.columns)]

['BsmtUnfSF', 'LotArea', 'Nbrhd_Area', 'GarageArea', 'GrLivArea', '1stFlrSF', 'TotalSF', 'Nbrhd_SF', 'Nbrhd_SFQC', 'LotFrontage', 'TotalBsmtSF', 'YearBuilt', 'BsmtFinSF1', 'YearRemodAdd', 'MoSold', 'GarageYrBlt', 'Nbrhd_QC', 'OpenPorchSF', 'WoodDeckSF', 'Yr_Sales']


[]

In [220]:
randomForest.set_params(n_estimators=100, random_state=42)
randomForest.fit(X_train.loc[:,Imp_feat], y_train)
print("The training error of random forest is: %.5f" %(1 - randomForest.score(X_train.loc[:,Imp_feat], y_train)))

The training error of random forest is: 0.00068


### Grid search on Random Forest

In [150]:
grid_para_forest = [{
    "n_estimators": [25, 50, 100],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int),
    "random_state": [42]}]
grid_search_forest = GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs=-1)
grid_search_forest.fit(X_train, y_train)

C:\Users\Gregory\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [25, 50, 100], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': range(1, 10), 'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]), 'random_state': [42]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [154]:
grid_search_forest.best_params_

{'criterion': 'gini',
 'min_samples_leaf': 4,
 'min_samples_split': 22,
 'n_estimators': 25,
 'random_state': 42}

In [155]:
#need to rerun, probably wrong score
grid_search_forest.best_score_

0.029452054794520548

#### Attempt with Logs

In [153]:
grid_search_forest2 = GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs=-1)
grid_search_forest2.fit(X_train.transform(np.log1p), y_train.transform(np.log1p))

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Gregory\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Gregory\Anaconda3\lib\runp

### Grid search with 20 best features from Random Forest

In [221]:
grid_search_forest_Impfeat = GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs=-1)
grid_search_forest_Impfeat.fit(X_train.loc[:,Imp_feat], y_train)

C:\Users\Gregory\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [25, 50, 100], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': range(1, 10), 'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]), 'random_state': [42]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [222]:
grid_search_forest_Impfeat.best_params_

{'criterion': 'gini',
 'min_samples_leaf': 3,
 'min_samples_split': 24,
 'n_estimators': 100,
 'random_state': 42}

In [223]:
grid_search_forest_Impfeat.best_score_

0.029452054794520548

In [227]:
randomForest.set_params(criterion='gini', min_samples_leaf=3, min_samples_split=24, n_estimators=1000, random_state=42)
rf_mod = randomForest.fit(X_train.loc[:,Imp_feat], y_train)
print("The training error of random forest is: %.5f" %(1 - randomForest.score(X_train.loc[:,Imp_feat], y_train)))

The training error of random forest is: 0.03219


In [228]:
score = ms.cross_val_score(rf_mod, X_train, y_train, cv=5)
score

C:\Users\Gregory\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


array([ 0.01912568,  0.02631579,  0.03465347,  0.03053435,  0.07692308])

In [234]:
y_pred_rF = rf_mod.predict(X_test.loc[:,Imp_feat])

In [239]:
y_pred_rF_df = pd.DataFrame(y_pred_rF, index = X_test.index, columns = ['SalePrice'])

In [244]:
y_pred_rF_df.set_index(y_pred_rF_df.index + 1).head()

,SalePrice
1461,120500
1462,165000
1463,173000
1464,181000
1465,181000


In [252]:
y_pred_rF_df.set_index(y_pred_rF_df.index + 1, inplace=True)
y_pred_rF_df = y_pred_rF_df.rename_axis('Id')

In [254]:
print(y_pred_rF_df.shape)
y_pred_rF_df.head()

(1459, 1)


,SalePrice
Id,
1461,120500
1462,165000
1463,173000
1464,181000
1465,181000


In [255]:
y_pred_rF_df.to_csv('submission_rF1.csv') #score of .2

## Stacked models

In [122]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(np.log(y), np.log(y_pred)))

In [142]:
models = [
    # linear model, ElasticNet = lasso + ridge
    ElasticNet(random_state=0, l1_ratio=.25, max_iter=2000),
    
    # conservative random forst model
    rfr(random_state=0,
        n_estimators=1000, max_depth=6,  max_features='sqrt'),
    
    # aggressive random forst model
    rfr(random_state=0, 
        n_estimators=1000, max_depth=9,  max_features='auto'),
    
    # conservative gbm model
    gbr(random_state=0, learning_rate = 0.005, max_features='sqrt',
        min_samples_leaf=15, min_samples_split=10, 
        n_estimators=3000, max_depth=3),
    
    # aggressive gbm model
    gbr(random_state = 0, learning_rate = 0.01, max_features='sqrt',
        min_samples_leaf=10, min_samples_split=5, 
        n_estimators = 1000, max_depth = 9)
    ]

meta_model = lr(normalize=True)

In [143]:
final_prediction = stacking_regression(models, meta_model, X_train, y_train, X_test,
                               transform_target=np.log1p, transform_pred = np.expm1, 
                               metric=rmsle, verbose=1)

metric: [rmsle]

model 0: [ElasticNet]


C:\Users\Gregory\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


    ----
    MEAN:   [0.19098342]

model 1: [RandomForestRegressor]
    ----
    MEAN:   [0.15062408]

model 2: [RandomForestRegressor]
    ----
    MEAN:   [0.14493062]

model 3: [GradientBoostingRegressor]
    ----
    MEAN:   [0.12707109]

model 4: [GradientBoostingRegressor]
    ----
    MEAN:   [0.12862033]

